In [19]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
import scipy as sp
from itertools import product
import sympy as sym

%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.4f}'.format})
GHz = 1e9

In [2]:
fluxonium = sq_ext.sq_fluxonium()
H_0_f = fluxonium.hamiltonian() 
H_0 = qt.tensor( H_0_f, qt.qeye(H_0_f.shape[0]) ) + qt.tensor(qt.qeye(H_0_f.shape[0]), H_0_f ) #+ 0.00001*qt.tensor(fluxonium.flux_op(0), fluxonium.flux_op(0))
H = qt.tensor( H_0_f, qt.qeye(H_0_f.shape[0]) ) + qt.tensor(qt.qeye(H_0_f.shape[0]), H_0_f  )+ 1e5*qt.tensor(fluxonium.flux_op(0), fluxonium.flux_op(0))

In [13]:
E, ψ_0_f = np.linalg.eigh(H_0_f.__array__() )
ψ_0 = np.array([np.kron(ψ_0_f[:,0],ψ_0_f[:,0]), np.kron(ψ_0_f[:,1],ψ_0_f[:,0]), np.kron(ψ_0_f[:,0],ψ_0_f[:,1]), np.kron(ψ_0_f[:,1],ψ_0_f[:,1]) ]).T

In [14]:
H_eff = ψ_0.T.conj() @ H.__array__() @ ψ_0 / 2 / np.pi / GHz
sq_ext.decomposition_in_pauli_4x4(H_eff, 4);

 (15.3596+0j)	*	 I ⨂I 
 (-0.9727+0j)	*	 I ⨂σz 
 (0.0235+0j)	*	 σx ⨂σx 
 (-0.9727+0j)	*	 σz ⨂I 


In [77]:
H, ψ_0 = sq_ext.hamiltonian_qubit_C_qubit(nmax_r=5, nmax_f=15, Cc=1, return_Ψ_nonint=True)

In [78]:
H_eff = sq_ext.H_eff_p1(ψ_0, H, 4, out='GHz', real=False, remove_ground = False)
sq_ext.decomposition_in_pauli_4x4(H_eff, 5); 

 (15.35777+0j)	*	 I ⨂I 
 (-0.95454+0j)	*	 I ⨂σz 
 (0.01687+0j)	*	 σy ⨂σy 
 (-0.95454+0j)	*	 σz ⨂I 


In [130]:
qubit_C_qubit = sq_ext.sq_qubit_C_qubit(Cc=1, nmax_r=5, nmax_f=15 )
H = qubit_C_qubit.hamiltonian()

In [131]:
H_eff = sq_ext.H_eff_p1(ψ_0, H.permute([2,0,3,1]), 4, out='GHz', real=False, remove_ground = False)
sq_ext.decomposition_in_pauli_4x4(H_eff, 5); 

 (16.83293+0j)	*	 I ⨂I 
 (-1.65871+0j)	*	 I ⨂σz 
 (-1.63884+0j)	*	 σz ⨂I 
 (0.36026+0j)	*	 σz ⨂σz 


In [72]:
Hs, ψ_0 = sq_ext.hamiltonian_qubit_C_qubit_C_qubit(nmax_r=5, nmax_f=5, Cc=0, return_Ψ_nonint=True, n_eig_Ψ_nonint=8, periodic=True)

In [49]:
n_eig_Ψ_nonint = 8
H_qubit, Ψ_q_0, E_q_0 = sq_ext.hamiltonian_qubit(nmax_r=5, nmax_f=15, return_Ψ_nonint=True, n_eig_Ψ_nonint=n_eig_Ψ_nonint)
E_q_C_q_C_q, Nq_Nq_Nq = sq_ext.generate_and_prioritize_energies([E_q_0, E_q_0, E_q_0], n_eig=n_eig_Ψ_nonint)
Ψ_0 = [qt.tensor([Ψ_q_0[Nq_Nq_Nq_i[0]], Ψ_q_0[Nq_Nq_Nq_i[1]],Ψ_q_0[Nq_Nq_Nq_i[2]]  ]) for Nq_Nq_Nq_i in Nq_Nq_Nq]

In [73]:
# Hf = sq_ext.sq_qubit(nmax_r=5, nmax_f=5).hamiltonian()
Hf = sq_ext.hamiltonian_qubit(nmax_r=5, nmax_f=5)
I = qt.qeye(Hf.dims[0])
Hs = qt.tensor(Hf, I, I) + qt.tensor(I, Hf, I) + qt.tensor(I, I, Hf)

In [74]:
Hs = sq_ext.hamiltonian_qubit_C_qubit(nmax_r=5, nmax_f=5, Cc=0)

In [75]:
ψ = sq_ext.diag(Hf, n_eig=2, solver='numpy')[1]
ψ0 = qt.Qobj(ψ[:,0])
ψ1 = qt.Qobj(ψ[:,1])

In [76]:
ψ_0 = [ qt.tensor(ψ0,ψ0,ψ0) ,
        qt.tensor(ψ1,ψ0,ψ0) ,
        qt.tensor(ψ0,ψ0,ψ0) ,
        qt.tensor(ψ0,ψ1,ψ0) ,
        qt.tensor(ψ0,ψ0,ψ0) ,
        qt.tensor(ψ0,ψ0,ψ1) ,
        qt.tensor(ψ1,ψ1,ψ0) ,
        qt.tensor(ψ1,ψ1,ψ1) ]
    

In [17]:
ψ_0[0].dag().__array__() @ Hs.__array__() @ ψ_0[0].__array__()

array([[1.25968355e+11+0.j]])

In [70]:
ψ_0 = [ qt.tensor(qt.Qobj(ψ[:,0]), qt.Qobj(ψ[:,0])), qt.tensor(qt.Qobj(ψ[:,1]), qt.Qobj(ψ[:,0])), qt.tensor(qt.Qobj(ψ[:,0]), qt.Qobj(ψ[:,1])), qt.tensor(qt.Qobj(ψ[:,1]), qt.Qobj(ψ[:,1]))]

In [77]:
H_eff = sq_ext.H_eff_p1(ψ_0=ψ_0, n_eig=8, H = Hs)

Wrong dimensions, doing the tensor product in dense form


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 625 is different from 15625)

In [65]:
sq_ext.decomposition_in_pauli_8x8(H_eff, 3); 

 (23.039+0j)	*	 I ⨂ I ⨂ I 
 (0.001+0j)	*	 I ⨂ I ⨂ σx 
 (-0.485+0j)	*	 I ⨂ I ⨂ σz 
 (0.001+0j)	*	 I ⨂ σx ⨂ I 
 (0.001+0j)	*	 I ⨂ σx ⨂ σx 
 (0.001+0j)	*	 I ⨂ σy ⨂ σy 
 (-0.485+0j)	*	 I ⨂ σz ⨂ I 
 (-0.001+0j)	*	 σx ⨂ I ⨂ I 
 (-0.001+0j)	*	 σx ⨂ I ⨂ σx 
 (-0.001+0j)	*	 σx ⨂ σx ⨂ I 
 (-0.001+0j)	*	 σx ⨂ σz ⨂ σz 
 (0.001+0j)	*	 σy ⨂ I ⨂ σy 
 (0.001+0j)	*	 σy ⨂ σy ⨂ I 
 (-1.456+0j)	*	 σz ⨂ I ⨂ I 
 (-0.001+0j)	*	 σz ⨂ σx ⨂ σz 
 (-0.001+0j)	*	 σz ⨂ σz ⨂ σx 
 (-0.485+0j)	*	 σz ⨂ σz ⨂ σz 


In [13]:
I_qubit = qt.identity(H_qubit.dims[0])
H_test =    (  qt.tensor(H_qubit, I_qubit, I_qubit)
               + qt.tensor(I_qubit, H_qubit, I_qubit)
               + qt.tensor(I_qubit, I_qubit, H_qubit) )
H == H_test

True

In [14]:
H_eff = sq_ext.H_eff_p1(ψ_0, H, 8, out='GHz', real=False, remove_ground = True)

In [28]:
Nq_Nq_Nq

[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1],
 [1, 1, 0],
 [1, 0, 1],
 [0, 1, 1],
 [1, 1, 1]]

In [25]:
E_q_C_q_C_q

array([0.0000, 1.8860, 1.8860, 1.8860, 3.7719, 3.7719, 3.7719, 5.6579])

In [27]:
sq_ext.diag(H_eff, 8, out = None, solver='numpy')[0]

array([0.0000, 1.8860, 1.8860, 1.8860, 3.7719, 3.7719, 3.7719, 5.6579])

In [15]:
sq_ext.decomposition_in_pauli_8x8(H_eff, 5);

 (2.82893+0j)	*	 I ⨂ I ⨂ I 
 (-0.47149+0j)	*	 I ⨂ I ⨂ σz 
 (-0.47149+0j)	*	 I ⨂ σz ⨂ I 
 (-1.41446+0j)	*	 σz ⨂ I ⨂ I 
 (-0.47149+0j)	*	 σz ⨂ σz ⨂ σz 


In [43]:
σz = qt.sigmaz()
σx = qt.sigmax()
I = qt.qeye(2)
# H_test = qt.tensor(σz, σz ,2*σx) + qt.tensor(I, σz, σz) + qt.tensor(I, I, σx) + qt.tensor(σx,σx,σx)
H_test = qt.tensor(σz,I, I) + qt.tensor(I, σz,I) + qt.tensor(I, I, σz) 
sq_ext.decomposition_in_pauli_8x8(H_test, 5);

 (1+0j)	*	 I ⨂ I ⨂ σz 
 (1+0j)	*	 I ⨂ σz ⨂ I 
 (1+0j)	*	 σz ⨂ I ⨂ I 


In [44]:
H_test

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[3.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000]
 [0.0000 1.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000]
 [0.0000 0.0000 1.0000 0.0000 0.0000 0.0000 0.0000 0.0000]
 [0.0000 0.0000 0.0000 -1.0000 0.0000 0.0000 0.0000 0.0000]
 [0.0000 0.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000]
 [0.0000 0.0000 0.0000 0.0000 0.0000 -1.0000 0.0000 0.0000]
 [0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 -1.0000 0.0000]
 [0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 -3.0000]]

In [39]:
H_test.__array__()[::-1,::-1]

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 3.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 3.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 5.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 3.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 5.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 5.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 7.+0.j]])

In [40]:
sq_ext.decomposition_in_pauli_8x8(H_test.__array__()[::-1,::-1], 5);

 (4+0j)	*	 I ⨂ I ⨂ I 
 (-1+0j)	*	 I ⨂ I ⨂ σz 
 (-1+0j)	*	 I ⨂ σz ⨂ I 
 (-1+0j)	*	 σz ⨂ I ⨂ I 


In [168]:
H_eff = sq_ext.H_eff_p1(ψ_0, H, 8, out='GHz', real=False, remove_ground = True, H_0 = H)
sq_ext.decomposition_in_pauli_8x8(H_eff, 5);

 (2.82874+0j)	*	 I ⨂ I ⨂ I 
 (-0.47146+0j)	*	 I ⨂ I ⨂ σz 
 (-0.47146+0j)	*	 I ⨂ σz ⨂ I 
 (-1.41437+0j)	*	 σz ⨂ I ⨂ I 
 (-0.47146+0j)	*	 σz ⨂ σz ⨂ σz 


In [188]:
np.round(np.real(H_eff),2)

array([[0.0000, 0.0000, 0.0000, 0.0000, -0.0200, -0.0200, -0.0200, 0.0000],
       [0.0000, 1.9100, 0.0200, 0.0200, 0.0000, 0.0000, 0.0000, -0.0200],
       [0.0000, 0.0200, 1.9100, 0.0200, 0.0000, 0.0000, 0.0000, -0.0200],
       [0.0000, 0.0200, 0.0200, 1.9100, 0.0000, 0.0000, 0.0000, -0.0200],
       [-0.0200, 0.0000, 0.0000, 0.0000, 3.8200, 0.0200, 0.0200, 0.0000],
       [-0.0200, 0.0000, 0.0000, 0.0000, 0.0200, 3.8200, 0.0200, 0.0000],
       [-0.0200, 0.0000, 0.0000, 0.0000, 0.0200, 0.0200, 3.8200, 0.0000],
       [0.0000, -0.0200, -0.0200, -0.0200, 0.0000, 0.0000, 0.0000, 5.7300]])